# Categorization
Contains word occurence analysis within the product name and description of the products and cleaning + analysis of the type column

## 1 Imports and Loads

In [1]:
import pandas as pd

In [2]:
products = pd.read_csv("data_clean/products_cl.csv")

In [3]:
# Create a copy
products_df = products.copy()

## 2 Type exploration and cleaning

In [15]:
products_df.head(5)

,sku,name,desc,listed_price,in_stock,type,man
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,1,8696,RAI
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,0,13855401,APP
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,0,1387,APP
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,0,1230,APP
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,1,1364,KIN


In [4]:
products_df.type.nunique()

114

In [ ]:
# Value counts reveal corrupted type entries (scientific notation , e.g. 5,49E+11 in German notation)
products_df.type.value_counts()

type
11865403     698
1298         354
11935397     318
11905404     282
12175397     226
            ... 
5,45E+15       1
113851714      1
12285400       1
113464259      1
113271716      1
Name: count, Length: 114, dtype: int64

In [ ]:
def clean_type_column(type_value):
    
    # Cleanin the 'type' column
    # Converting scientific notation into normal integer.
    # Removing commas and make at most 9 digits.
    # If the number is longer than 9 digits, it reduces it to the first 9 digits.
    # Returns--> A cleaned integer or string if conversion is not possible
    try:
        num = float(str(type_value).replace(',', '.'))
        
        num_str = str(int(num))[:9]
        
        return int(num_str)
    
    except ValueError:
        return type_value  
    # If conversion fails return original value (string)

In [8]:
# Apply the cleaning function to the 'type' column
products_df['type'] = products_df['type'].apply(clean_type_column)
products_df.type.value_counts()

type
11865403     698
1298         354
11935397     318
11905404     282
12175397     226
            ... 
545000000      1
113851714      1
12285400       1
113464259      1
113271716      1
Name: count, Length: 114, dtype: int64

In [9]:
type_num_list = products_df['type'].unique().tolist()
type_num_list

[8696,
 13855401,
 1387,
 1230,
 1364,
 1325,
 1334,
 13005399,
 13835403,
 12995397,
 13955395,
 1276,
 11905404,
 12635403,
 11865403,
 12755395,
 12355400,
 1229,
 11935397,
 12655397,
 1404,
 101781405,
 12085400,
 1424,
 9094,
 14305406,
 10142,
 12645406,
 10230,
 12215397,
 14365395,
 5384,
 12175397,
 57445397,
 5395,
 13555403,
 21485407,
 20642062,
 1280,
 12575403,
 1405,
 14035403,
 1433,
 1216,
 1515,
 12585395,
 1392,
 1231,
 549000000,
 5398,
 13615399,
 1296,
 1375,
 5405,
 1298,
 1282,
 12285400,
 144000000,
 5720,
 15435404,
 11821715,
 24215399,
 11434,
 2425,
 2434,
 2449,
 42945397,
 13621714,
 24861714,
 24821716,
 54864259,
 4259,
 51861714,
 1416,
 51601716,
 51871714,
 12141714,
 5403,
 54085407,
 21535407,
 5407,
 5406,
 5404,
 544000000,
 85641716,
 42931714,
 24811716,
 85651716,
 24885185,
 24895185,
 54025401,
 543000000,
 5401,
 1714,
 572000000,
 1716,
 21622158,
 51882158,
 217000000,
 12031714,
 12051714,
 21571716,
 106431714,
 5399,
 21561716,
 79201

In [12]:
# Explore for specific types
mask = products_df['type'] == 144000000
products_df.loc[mask, ['name', 'type']]

,name,type
628,iPhone 5s battery repair,144000000
649,Full Screen Repair iPhone 6,144000000
823,battery repair iPhone 6,144000000
824,Home button repair iPhone 6,144000000
825,Load Connector Repair iPhone 6,144000000
826,GSM antenna repair iPhone 6,144000000
1706,Full screen repair iPhone 5s,144000000
3288,Load Connector Repair iPhone 5,144000000
3289,Full screen repair iPhone 6s,144000000
3290,Full screen repair iPhone 6s Plus,144000000


## 3 Explore connection between type and product name

In [16]:
def find_type(type_num, df):
    # Filter rows where 'type' matches the given type_num 
    mask_type = df['type'] == type_num
    column_texts = df.loc[mask_type, 'name']
    
    # Convert the column to a single lowercase string 
    all_text = ' '.join(column_texts.astype(str)).lower()
    words = all_text.split() #split into words

    # words to skip
    # you can change the words to skip
    skip_words = {'and', 'for', 'with', 'of', 'to', 'apple', 'pro', 'air', 'silver', 'second', 'hand'}
    
    # filter the words
    #non-alphabetic words and words in skip_words
    filtered_words = []
    for word in words:
        if word.isalpha() and word not in skip_words:
            filtered_words.append(word)
    
    # Count the occurrences
    word_counts = {}
    for word in filtered_words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    # top most common words
    # you can change the number in range --> how many words you want to display
    most_common_words = []
    for i in range(1):
        max_word = ''
        max_count = 0
        for word, count in word_counts.items():
            if count > max_count:
                max_word = word
                max_count = count
        if max_word:
            most_common_words.append(max_word)
            # Remove it from the initial dictionary
            #to find the next most common one
            del word_counts[max_word]  

    return most_common_words

In [ ]:
# Apply lingual analysis and create a new list with most common words to assist in category forming
category_list = [find_type(item, products_df) for item in type_num_list]
category_list

[['support'],
 ['keyboard'],
 ['mouse'],
 ['cable'],
 ['mac'],
 ['cable'],
 ['gigabit'],
 ['macbook'],
 ['macbook'],
 ['usb'],
 ['like'],
 ['pcie'],
 ['white'],
 ['ipad'],
 ['case'],
 ['owc'],
 ['adapter'],
 ['pointer'],
 ['usb'],
 ['drive'],
 ['synology'],
 ['wacom'],
 ['stick'],
 ['usb'],
 ['camera'],
 ['ifixit'],
 ['battery'],
 ['ifixit'],
 ['macbook'],
 ['ssd'],
 ['adapter'],
 ['headphones'],
 ['nas'],
 ['card'],
 ['allocacoc'],
 ['iphone'],
 ['iphone'],
 ['installation'],
 ['mini'],
 ['keyboard'],
 ['wacom'],
 ['ipad'],
 ['kit'],
 ['ipad'],
 ['battery'],
 ['usb'],
 ['backpack'],
 ['applecare'],
 ['battery'],
 ['speaker'],
 ['charger'],
 ['monitor'],
 ['blue'],
 ['sport'],
 ['open'],
 ['ram'],
 ['whoosh'],
 ['repair'],
 ['iphone'],
 ['leap'],
 ['ipod'],
 ['watch'],
 ['security'],
 ['screen'],
 ['watch'],
 ['strap'],
 ['lightning'],
 ['ipad'],
 ['ipad'],
 ['iphone'],
 ['remote'],
 ['tv'],
 ['ipad'],
 ['mac'],
 ['iphone'],
 ['ipad'],
 ['ipad'],
 ['case'],
 ['ifixit'],
 ['ifixit'],
 [

In [23]:
# Take another approach to scout specific types and their statistics
def cat_search(type, df):
  list_1 = df.loc[df["type"] == type, ["name", "desc"]].sort_values(by="name")
  stat_1 = df.loc[df["type"] == type].describe()
  return list_1, stat_1

In [24]:
cat_search(13855401, products_df)

(                                                   name  \
 3528  (OEM) Apple Wireless Keyboard Mac Keyboard Eng...   
 1166    Apple Keyboard Keypad International English Mac   
 1                     Apple Mac Keyboard Keypad Spanish   
 3734  Apple Magic Spanish Keyboard Keypad Bluetooth Mac   
 1948      Belkin Bluetooth Keypad YourType aluminum Mac   
 20                      KeyPad LMP Bluetooth Keypad Mac   
 2079  LMP Bluetooth Keypad 2 Numeric Keypad for Magi...   
 4738       Like new - Apple Mac Keyboard Keypad Spanish   
 3639                 Like new - Apple Wireless Keyboard   
 3530  Logitech Desktop MK120 Keyboard and USB Mouse ...   
 1327  Logitech Keyboard K380 Multi-Device Black for ...   
 2343  Logitech Wireless Keyboard K780 Black Multi De...   
 1999  Macally Keyboard Magic Keyboard Protector Tran...   
 3983  Macally Keyboard Protector TPU MacBook Pro Tou...   
 3410  Matias Bluetooth keyboard Aluminum + Spanish n...   
 4704   Matias numeric keyboard + Spanis

In [100]:
cat_search(5395, products_df)

(                                                   name  \
 201      Adapter Apple Lightning to Micro USB connector   
 125           Adapter Apple Lightning to SD Card Reader   
 3541    Allocacoc PowerCube Extended Remote Power strip   
 3540  Allocacoc PowerCube Extended USB plug strip Wh...   
 3539  Allocacoc PowerCube Extended USB plug strip Wh...   
 4159  Allocacoc PowerCube Extended USB plug strip Wh...   
 4160  Allocacoc PowerCube Extended USB plug strip Wh...   
 3537  Allocacoc PowerCube Extended plug strip White ...   
 4174  Allocacoc PowerCube Original Fluorescent lamp ...   
 4173  Allocacoc PowerCube Original Regleta White / Blue   
 4171  Allocacoc PowerCube Original Regleta White / G...   
 4172   Allocacoc PowerCube Original Regleta White / Red   
 4157  Allocacoc PowerCube Original USB Fluorescent l...   
 4158  Allocacoc PowerCube Original USB Fluorescent l...   
 3535  Allocacoc PowerCube Original USB wall Regleta ...   
 3538  Allocacoc PowerCube extended plug

## 4 Category assignment

In [25]:
# add category to the DataFrame
products_df["category"] = ""

In [26]:
category_type_df = products_df.copy()
n = 50
print(f"With the {n} largest types, we account for {((category_type_df.groupby('type').count().nlargest(n, 'sku')['sku'].sum()) / (category_type_df.shape[0]) * 100).round(2)}% of all products.")

With the 50 largest types, we account for 91.05% of all products.


Algorithm assisted procedure results in determining the following categories:
- Cases & Protectors (Cases / Protectors)
- Storage Devices (Hard Drives, Data Storage, Memory Card/USB) – Now includes Computer Memory
- Laptops & Computers (Laptops, Computers)
- Accessories & Peripherals (Accessories, Adaptors/Cables, Stands/Mounts)
- Repair & Maintenance (Repair Kits, Open/Second Hand - Misc)
- Monitors & Display (Monitor)
- Audio & Sound (Headphones, Speakers)
- Smartwatches & Accessories (Smart Watch, Smart Watch Straps, Smart Watch Chargers)
- Mobile Phones & Accessories (Phones, Chargers)
- Tablets & E-Readers (Tablets, Smartpen)
- Cameras & Photography (Cameras, iPods)
- Power & Batteries (Batteries/PB)
- Open & Second Hand - Misc
- Networking Devices
- Backpacks
- Miscellaneous (Any remaining uncategorized items)

In [114]:
products_df.loc[products_df["type"] == 11865403, "category"] = "Cases & Protectors" 
products_df.loc[products_df["type"] == 12175397, "category"] = "Storage Devices" 
products_df.loc[products_df["type"] == 1298, "category"] = "Open & Second Hand - Misc"
products_df.loc[products_df["type"] == 1716, "category"] = "Open & Second Hand - Misc" 
products_df.loc[products_df["type"] == 11935397, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 1276, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 11905404, "category"] = "Accessories & Peripherals"
products_df.loc[products_df["type"] == 5395, "category"] = "Accessories & Peripherals" 
products_df.loc[products_df["type"] == 1282, "category"] = "Laptops & Computers" 
products_df.loc[products_df["type"] == 12635403, "category"] = "Cases & Protectors"
products_df.loc[products_df["type"] == 13835403, "category"] = "Cases & Protectors" 
products_df.loc[products_df["type"] == 574000000, "category"] = "Laptops & Computers" 
products_df.loc[products_df["type"] == 1364, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 1296, "category"] = "Monitors & Display"
products_df.loc[products_df["type"] == 1325, "category"] = "Accessories & Peripherals"
products_df.loc[products_df["type"] == 1387, "category"] = "Accessories & Peripherals"
products_df.loc[products_df["type"] == 12585395, "category"] = "Accessories & Peripherals"
products_df.loc[products_df["type"] == 5384, "category"] = "Audio & Sound"
products_df.loc[products_df["type"] == 1433, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 12215397, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 5398, "category"] = "Audio & Sound"
products_df.loc[products_df["type"] == 102000000, "category"] = "Laptops & Computers"
products_df.loc[products_df["type"] == 144000000, "category"] = "Repair & Maintenance"
products_df.loc[products_df["type"] == 57445397, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 1334, "category"] = "Networking Devices"
products_df.loc[products_df["type"] == 2158, "category"] = "Laptops & Computers"
products_df.loc[products_df["type"] == 2449, "category"] = "Smartwatches & Accessories"
products_df.loc[products_df["type"] == 12655397, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 1229, "category"] = "Tablets & E-Readers"
products_df.loc[products_df["type"] == 12995397, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 1515, "category"] = "Power & Batteries"
products_df.loc[products_df["type"] == 13615399, "category"] = "Mobile Phones & Accessories"
products_df.loc[products_df["type"] == 13555403, "category"] = "Cases & Protectors"
products_df.loc[products_df["type"] == 1405, "category"] = "Tablets & E-Readers"
products_df.loc[products_df["type"] == 1230, "category"] = "Accessories & Peripherals"
products_df.loc[products_df["type"] == 118692158, "category"] = "Laptops & Computers"
products_df.loc[products_df["type"] == 1216, "category"] = "Accessories & Peripherals"
products_df.loc[products_df["type"] == 24885185, "category"] = "Smartwatches & Accessories"
products_df.loc[products_df["type"] == 24895185, "category"] = "Smartwatches & Accessories"
products_df.loc[products_df["type"] == 21485407, "category"] = "Repair & Maintenance"
products_df.loc[products_df["type"] == 1392, "category"] = "Backpacks"
products_df.loc[products_df["type"] == 11821715, "category"] = "Cameras & Photography"
products_df.loc[products_df["type"] == 8696, "category"] = "Accessories & Peripherals"
products_df.loc[products_df["type"] == 9094, "category"] = "Cameras & Photography"
products_df.loc[products_df["type"] == 539000000, "category"] = "Laptops & Computers"
products_df.loc[products_df["type"] == 549000000, "category"] = "Power & Batteries"
products_df.loc[products_df["type"] == 13005399, "category"] = "Power & Batteries"
products_df.loc[products_df["type"] == 42945397, "category"] = "Storage Devices"
products_df.loc[products_df["type"] == 12645406, "category"] = "Repair & Maintenance"
products_df.loc[products_df["type"] == 51601716, "category"] = "Mobile Phones & Accessories"
products_df.loc[products_df["type"] == 216000000, "category"] = "Laptops & Computers"
products_df.loc[products_df["type"] == 85651716, "category"] = "Mobile Phones & Accessories"
products_df.loc[products_df["type"] == 24215399, "category"] = "Smartwatches & Accessories"
products_df.loc[products_df["type"] == 5720, "category"] = "Accessories & Peripherals"
products_df.loc[products_df["type"] == 14305406, "category"] = "Repair & Maintenance"

In [115]:
products_df['category'].value_counts()

category
Storage Devices                1179
Cases & Protectors             1108
Accessories & Peripherals       761
                                540
Open & Second Hand - Misc       357
Audio & Sound                   220
Smartwatches & Accessories      144
Mobile Phones & Accessories     130
Monitors & Display              121
Tablets & E-Readers             119
Repair & Maintenance            111
Power & Batteries                91
Networking Devices               74
Cameras & Photography            46
Laptops & Computers              32
Backpacks                        28
Name: count, dtype: int64

In [116]:
# Check the NaN values 
# Create a dataframe having only Nan values in the category column   
mask_null = products_df['category'] == ''
no_cat_df = products_df.loc[mask_null, :]
no_cat_df
# mask_null.value_counts()

,sku,name,desc,listed_price,in_stock,type,man,category
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,0,13855401,APP,
15,APP0234,Apple Dock Connector to VGA,Dock Connector to VGA IOS.,35.00,0,13955395,APP,
20,LMP0007,KeyPad LMP Bluetooth Keypad Mac,expansion keypad Mac with bluetooth connection.,54.99,1,13855401,LMP,
25,OWC0007,OWC Data Doubler Optical Bay adapter Macbook /...,Tray SuperDrive replacement hard disk or SSD.,72.99,1,12755395,OWC,
32,MOS0065,Moshi TeraGlove cleaner screens,Microfibre screen cleaner for Mac iPhone iPad ...,20.00,0,12355400,MOS,
...,...,...,...,...,...,...,...,...
4992,AP20455,Like new - Apple iPhone 8 256GB Gold,Apple iPhone 8 reconditioned 256GB in Gold rea...,979.00,0,113291716,AP2,
4996,ZAG0026-A,Open - Zagg Rugged Keyboard Folio iPad Messeng...,Case reconditioned keyboard and adjustable pos...,99.99,0,12575403,ZAG,
5024,THU0059,Subterra Thule Attache Case MacBook Pro / Reti...,zipper bag and shoulder bag for MacBook Pro / ...,79.99,0,10230,THU,
5027,SDE0007,SDesign iPhone charging dock QI X / 8 / Apple ...,Base load compatible with iPhone and Apple Wat...,69.99,0,5399,SDE,


In [ ]:
# Assigning categories based on keywords in the 'desc' or 'name' column
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("battery|charger", case=False), "category"] = "Power & Batteries"
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("adapter|cable|support|connector|stand|keyboard|mouse|accessory", case=False), "category"] = "Accessories & Peripherals"
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("protector|case", case=False), "category"] = "Cases & Protectors"
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("microphone", case=False), "category"] = "Audio & Sound"
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("watch|strap", case=False), "category"] = "Smartwatches & Accessories"
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("ipad", case=False), "category"] = "Tablets & E-Readers"
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("gopro", case=False), "category"] = "Cameras & Photography"
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("like new|second hand", case=False), "category"] = "Open & Second Hand - Misc"

In [ ]:
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("battery|charger", case=False), "category"] = "Power & Batteries"
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("adapter|cable|support|connector|stand|keyboard|mouse|accessory", case=False), "category"] = "Accessories & Peripherals"
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("protector|case", case=False), "category"] = "Cases & Protectors"
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("microphone", case=False), "category"] = "Audio & Sound"
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("watch|strap", case=False), "category"] = "Smartwatches & Accessories"
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("ipad", case=False), "category"] = "Tablets & E-Readers"
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("gopro", case=False), "category"] = "Cameras & Photography"
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("like new|second hand", case=False), "category"] = "Open & Second Hand - Misc"

In [119]:
#Check how many Nan values left
no_cat_df.loc[no_cat_df['category'] == '', :].sort_values(by='type')

,sku,name,desc,listed_price,in_stock,type,man,category
655,SYN0107,Synology RackStation RX415 expansion module,Expansion Unit 4-bay Synology NAS server.,544.99,0,1280,SYN,
3523,DRO0030,5N2 Drobo NAS server Mac and PC,5-bay NAS server with two Gigabit Ethernet por...,689.00,0,1280,DRO,
3818,QNA0228,QNAP QM2-2P PCIe expansion card dual M.2 SSD,PCIe expansion card to connect two M.2 SSD,136.23,0,1404,QNA,
408,SYN0093,Synology Pack 8 IP camera license,8 pack license Mac and PC camera.,329.99,0,1404,SYN,
3959,SYN0172,Synology DX517 expansion unit NAS,expansion module for Nas eSATA and 5 bays,517.99,0,1404,SYN,
...,...,...,...,...,...,...,...,...
4145,APP2477,Apple iPhone 64GB Space Gray 8,Apple iPhone 64GB 8 Color Space Gray,809.00,1,113291716,APP,
4149,APP2481,Apple iPhone 8 256GB Silver,256GB Apple iPhone 8 in Silver,979.00,0,113291716,APP,
4925,APP2487-A,Open - 32GB Apple TV 4K,Apple multimedia player with open box,199.00,0,113464259,APP,
3991,AP20296,"Apple MacBook Air 13 ""Core i5 16GHz | 4GB RAM ...",Computer Refurbished MacBook Air 13-inch i5 16...,1099.00,0,217000000,AP2,


In [ ]:
# The following cells help assign the remaining products
no_cat_df.loc[no_cat_df["desc"].str.lower().str.contains("accessory", case=False), :]

,sku,name,desc,listed_price,in_stock,type,man,category
53,WAC0055,Wacom Intuos Professional Accessory Kit,Intuos accessory kit.,24.9,0,101781405,WAC,


In [69]:
no_cat_df.loc[no_cat_df["name"].str.lower().str.contains("accessory", case=False), :]

,sku,name,desc,listed_price,in_stock,type,man,category
41,WAC0040,Wacom Bamboo Wireless Kit / Intuos graphics ta...,Wireless Kit Wacom graphics tablets.,39.99,1,101781405,WAC,
53,WAC0055,Wacom Intuos Professional Accessory Kit,Intuos accessory kit.,24.90,0,101781405,WAC,


In [ ]:
# Locating certain categories for consistency checks
products_df.loc[products_df["category"] == "Networking Devices", :].sort_values(by='type')

,sku,name,desc,listed_price,in_stock,type,man,category
9,APP0111,Apple USB Ethernet Adapter Mac,USB to Ethernet adapter Mac.,35.00,0,1334,APP,Networking Devices
2233,TPL0010,TP-Link Wireless USB Adapter T2U Archer Mac Wi...,Pendrive USB 2.0 WiFi adapter AC600 double ban...,27.71,0,1334,TPL,Networking Devices
2232,TPL0024,TP-Link TL-MR6400 Router 4G Wi-Fi N 300Mbps,N wifi router (24GHz) 300Mbps wireless speed,156.09,0,1334,TPL,Networking Devices
2231,TPL0015,TP-Link Gigabit Router Archer C7 Wi-Fi ac 1750...,Wireless Gigabit Router AC1750 Dual Band iPhon...,131.89,0,1334,TPL,Networking Devices
2230,TPL0012,TP-Link Gigabit Router C3200 Archer Wi-Fi ac 3...,Router Gigabit Wireless Router for Mac Tri-Ban...,337.59,0,1334,TPL,Networking Devices
...,...,...,...,...,...,...,...,...
456,DLK0078,D-Link DGS-1008D Gigabit Switch 8 ports,D-Link Switch with 8 Gigabit ports for high pe...,29.90,0,1334,DLK,Networking Devices
455,DLK0076,D-Link DES-1016D Switch 16 10 / 100Mbps unmanaged,D-Link DES-1016D with 16 ports for group work.,59.99,0,1334,DLK,Networking Devices
454,DLK0075,D-Link GO-SW-8E Switch 8 Ethernet ports,D-Link Switch 8 Ethernet ports maximum speed.,18.99,0,1334,DLK,Networking Devices
452,DLK0085,D-Link DWA-140 WiFi-N USB mini plug,DLink Wireless 802.11N USB mini plug with spee...,17.99,0,1334,DLK,Networking Devices


In [ ]:
# Locating certain types for consistency checks
no_cat_df.loc[no_cat_df["type"] == 217000000, :].sort_values(by='type')

,sku,name,desc,listed_price,in_stock,type,man,category
3565,AP20085,"Like new - Apple Macbook Air 13 ""i5 16GHz | 8G...",Computer Refurbished MacBook Air 13-inch i5 16...,1349.00,0,217000000,AP2,Open & Second Hand - Misc
3568,AP20084,"Like new - Apple Macbook Air 13 ""i5 16GHz | 8G...",Computer Refurbished MacBook Air 13-inch i5 16...,1099.00,0,217000000,AP2,Open & Second Hand - Misc
3729,APP2072,"Apple MacBook Air 13 ""Core i5 18GHz | 8GB RAM ...",laptop MacBook Air 13 inch i5 18GHz 8GB RAM an...,1105.59,1,217000000,APP,
3923,AP20236,"Like new - Apple MacBook Air 13 ""Core i5 16GHz...",Computer Refurbished MacBook Air 13-inch i5 16...,1099.00,0,217000000,AP2,Open & Second Hand - Misc
3991,AP20296,"Apple MacBook Air 13 ""Core i5 16GHz | 4GB RAM ...",Computer Refurbished MacBook Air 13-inch i5 16...,1099.00,0,217000000,AP2,
4595,AP20391,"Like new - Apple MacBook Air 13 ""1.8GHz Core i...",13-inch MacBook Air Refurbished 1.8GHz dual-co...,1105.59,0,217000000,AP2,Open & Second Hand - Misc


In [120]:
no_cat_df.loc[no_cat_df['category'] == '', 'category'] = 'Miscellaneous'

In [122]:
# update the original dataframe with new category values
products_df.update(no_cat_df)

In [127]:
products_df['category'].value_counts()

category
Storage Devices                1179
Cases & Protectors             1143
Accessories & Peripherals       820
Open & Second Hand - Misc       479
Tablets & E-Readers             227
Audio & Sound                   225
Smartwatches & Accessories      176
Miscellaneous                   143
Mobile Phones & Accessories     130
Power & Batteries               125
Monitors & Display              121
Repair & Maintenance            111
Networking Devices               74
Cameras & Photography            48
Laptops & Computers              32
Backpacks                        28
Name: count, dtype: int64

## 5 Export

In [129]:
products_df.to_csv("data_clean/products_cl.csv", index=False)